Link to Github repository: https://github.com/fridamariejoergensen/CSS.git <br>
### Contribution Statement

We, Frida(s206182), Cecilie(sXXXXXX), and Marie(sXXXXXX), collaborated on Assignment 1 in Jupyter Notebook and made the following contributions:

- Cecilie focused on this

- Frida did this

- Marie did this



# Assignment 1

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import PyPDF2
import pickle

ModuleNotFoundError: No module named 'pandas'

## Part 1: Using web-scraping to gather data

### 2019 edition

In [ ]:
## Poster
LINK = "https://2019.ic2s2.org/posters/"
r = requests.get(LINK) 
soup = BeautifulSoup(r.content)

In [ ]:
listul = soup.find("div",{"class":"col-md-8 page-content-wrap"})
listli = listul.find_all("li")

allnames = []
for i in range(len(listli)):
    if listli[i].find("strong"):
        listli[i].strong.extract()
    if listli[i].find("span"):
        listli[i].span.extract()
    if listli[i].find("br"):
        listli[i].br.extract()
        
    names = listli[i].text.replace("\n","").replace(" and ",",").replace(", ",",")
    allnames = allnames + names.split(",")
    


un2019 = np.unique(np.array(allnames))
len(un2019)

There was ?? unique researchers that made poster presentations in 2019  

In [6]:
#### Oral
LINK = 'https://2019.ic2s2.org/oral-presentations/'
r = requests.get(LINK)
all_oral = BeautifulSoup(r.content)

In [ ]:
# extract the specific element of the site that contains the names
paragraphs = all_oral.find_all("p", {"class":""})[3:39]

In [ ]:
names = []
for p in paragraphs:
    text = p.text.strip()
    lines = text.split(' – ')
    names.extend([line.split('.')[0] for line in lines[1:]])

In [ ]:
# code that extracts list elements that contain numbers.

def remove_elements_with_numbers(names):
    return [i for i in names if not any(j.isdigit() for j in i)]

p = remove_elements_with_numbers(names)

In [2]:
newlist = [word for line in p for word in line.split(',')]
newlist.remove('No Presentation')
final_list = np.asarray(newlist)
print(len(np.unique(final_list)))

NameError: name 'p' is not defined

There was ?? unique researchers that made oral presentations in 2019  

In [5]:
# Combine the two lists of poster and oral presentators
combined_list = np.concatenate((final_list, un2019))
# Find the length of unique combined list that attended the conference in 2019 
print(len(np.unique(combined_list)))

NameError: name 'np' is not defined

In total there was ?? unique researchers you got in 2019

Explain one or two decisions you took during the web-scraping exercise, for 2019 or any other year. Why did you take this choice? How might your decision impact the final number of authors?

## Part 2: Getting data from the Semantic Scholar API
Consider the list of author ids you have found in Week 2, Part 3, first excercise. For each author, use the Academic Graph API to find: <br>
- their aliases <br>
- their name <br>
- their papers <br>

(**Share the number of authors you will use as starting point in this exercises. Add a comment clarifying how many IC2S2 editions you included and if the collaborators were included or not!!)

Create three dataframe to store the data you have collected:

**Author dataset**: in the author dataset, one raw is one unique author, and each row contains the following information: <br>
- authorId: (str) the id of the author <br>
- name: (str) the name of the author <br> 
- aliases: (list) the aliases of the author <br>
- citationCount: (int) the total number of citations received by an author <br>
- field: (str) the s2FieldsOfStudy that occurs most times across an author's papers (you should first obtain the category for each s2FieldsOfStudy) 



**Paper dataset**: in the paper dataset, one row is one unique paper, and each row contains the following information: <br>
- paperId: (str) the id of the paper <br>
- title: (str) the title of the paper <br> 
- year: (int) the year of publication <br>
- externalId.DOI: (str) the DOI of the paper <br>
- citationCount: (int) the number of citations <br>
- fields: (list) the fields included in the paper (you should first obtain the category for each s2FieldsOfStudy) <br>
-  authorIds: (list) this is a list of author Ids, including all the authors of this paper that are in our author dataset



**Paper abstract dataset**: in the paper abstract dataset, one row is one unique paper, and each row contains the following information: <br>
- paperId: (str) the id of the paper <br>
- abstract: (str) the abstract of the paper


How long is your final Author dataframe? How long is your final Paper dataframe?

## Part 3: Law of large numbers.